In [1]:
import os
import asyncio
from dotenv import load_dotenv
from sf_primary_agent import SFPrimaryAgent
from sf_researcher.context.pinecone_utils import PineconeManager
from sf_researcher.memory import Memory
from sf_researcher.master.functions import construct_contacts
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [3]:
request = {
    "query": "OpenSea",
    "contacts": [],
    "include_domains": [],
    "exclude_domains": [],
    "parent_sub_queries": [
        "\"OpenSea\" company registered directors"
    ],
    "child_sub_queries": [
        "director data"
    ],
    "salesforce_id": "00QQy000007D0hKMAS",
    "parent_retreiver_queries": [
        "OpenSea directors"
    ],
    "child_retreiver_queries": [
        "director data"
    ]
}
main_report_type="compliance_report"
child_report_type="contact_report"

In [4]:
researcher = SFPrimaryAgent(
        query=request["query"],
        namespace=request["salesforce_id"],
        source_urls=None,
        config_path="",
        contacts=request["contacts"],
        include_domains=request["include_domains"],
        exclude_domains=request["exclude_domains"],
        main_report_type=main_report_type,
        child_report_type=child_report_type,
        parent_sub_queries=request["parent_sub_queries"],
        child_sub_queries=request["child_sub_queries"],
        parent_retreiver_queries=request["parent_retreiver_queries"],
        child_retreiver_queries=request["child_retreiver_queries"]
    )

INFO:sf_primary_agent:No contacts provided. Contacts will be constructed.


In [5]:
await researcher.main_task_assistant.conduct_research_insert()

🔎 Starting the research task for 'OpenSea'...
Compliance Agent


INFO:sf_researcher.master.agent:
🔧 get_new_urls results:
['https://en.wikipedia.org/wiki/OpenSea', 'https://opensea.io/blog/articles/welcoming-chris-dixon-to-our-board-of-directors', 'https://www.datanyze.com/companies/opensea/452495898', 'https://www.forbes.com/companies/opensea/', 'https://www.cbinsights.com/company/opensea/people']



✅ Added source url to research: https://en.wikipedia.org/wiki/OpenSea

✅ Added source url to research: https://opensea.io/blog/articles/welcoming-chris-dixon-to-our-board-of-directors

✅ Added source url to research: https://www.datanyze.com/companies/opensea/452495898

✅ Added source url to research: https://www.forbes.com/companies/opensea/

✅ Added source url to research: https://www.cbinsights.com/company/opensea/people

🤔 Scraping sites for relevant information...



ERROR:sf_researcher.scraper.beautiful_soup.beautiful_soup:❌ beautiful_soup.py Failed to fetch URL: https://www.datanyze.com/companies/opensea/452495898, Status code: 403
 content: 

INFO:sf_researcher.master.agent:
🌐 scrape_urls content results for sub-query '"OpenSea" company registered directors':
 [{'url': 'https://en.wikipedia.org/wiki/OpenSea', 'raw_content': 'Contents\nOpenSea\nOpenSea is an American non-fungible token (NFT) marketplace headquartered in New York City. The company was founded by Devin Finzer and Alex Atallah in 2017.[2][3]\nOpenSea offers a marketplace online allowing for non-fungible tokens to be sold directly at a fixed price, or through an auction.\nHistory[edit]\nDevin Finzer and Alex Atallah founded OpenSea in December 2017.[4] They were inspired by CryptoKitties, a blockchain-based game featuring non-fungible tokens that had been released earlier that year.[4] Finzer and Atallah believed that OpenSea could be a marketplace for buying NFT tokens like those.[4

{'num_added': 8, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}


In [6]:
await researcher.main_task_assistant.conduct_research_query(researcher.parent_retreiver_queries)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:sf_researcher.master.agent:
🧩 agent.py conduct_research_query query: OpenSea directors:



🌐 Document source: https://www.cbinsights.com/company/opensea/people
📑 Document content: Wells Fargo Success Story
Learn More →
Professional Services
Learn More →
Platform Overview
Learn More →
Loading...
OpenSea
Executives
OpenSea Management Team
5 Team Members
OpenSea has 5 executives. OpenSea's
current Founder, Chief Executive Officer is Devin
Finzer.
Name
Work History
Title
Status
Devin
Finzer
Credit Karma, Pinterest, Flipboard, Google, and Wikimedia Foundation
Founder, Chief Executive Officer
Current
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Name
Devin
Finzer
Subscribe to see more
Subscribe to see more
Subscribe to see more
Subscribe to see more
Work History
Credit Karma, Pint

INFO:sf_researcher.master.agent:
🧩 Aggregated self.context: ["Wells Fargo Success Story\nLearn More →\nProfessional Services\nLearn More →\nPlatform Overview\nLearn More →\nLoading...\nOpenSea\nExecutives\nOpenSea Management Team\n5 Team Members\nOpenSea has 5 executives. OpenSea's\ncurrent Founder, Chief Executive Officer is Devin\nFinzer.\nName\nWork History\nTitle\nStatus\nDevin\nFinzer\nCredit Karma, Pinterest, Flipboard, Google, and Wikimedia Foundation\nFounder, Chief Executive Officer\nCurrent\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nName\nDevin\nFinzer\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nSubscribe to see more\nWork History

In [7]:
# Update context of the global context variable
researcher.global_context = researcher.main_task_assistant.context
# Update url list of the global list variable
researcher.global_urls = researcher.main_task_assistant.visited_urls

In [8]:
# Get list of all subtopics
if not researcher.contacts:
    logger.info(f"\n🕵️‍♂️ Constructing contacts\n")
    researcher.contacts = await construct_contacts(
        company_name=researcher.query,
        data=researcher.main_task_assistant.context,
        config=researcher.main_task_assistant.cfg,
    )
else:
    logger.info("\n🕵️‍♂️ Using the provided contacts list\n")

logger.info(f"\n🕵️‍♂️ Contact Search List: {researcher.contacts}\n")

INFO:__main__:
🕵️‍♂️ Constructing contacts




🤖 llm.py construct_contacts Calling gpt-3.5-turbo...



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:
🕵️‍♂️ Contact Search List: contacts=[Contact(first_name='Chris', last_name='Dixon'), Contact(first_name='Katie', last_name='Haun')]

